In [1]:
import turicreate as tc
import os
import re
import MeCab

# MeCab 오브젝트 생성
tagger = MeCab.Tagger("-Owakati")

/Users/furukawahidekazu/anaconda3/envs/turicreate/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/furukawahidekazu/anaconda3/envs/turicreate/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# 파일 읽어오기
def read_file(path):
    files = os.listdir(path)
    array = []
    for file in files:
        if file.startswith('.'):
            continue
        file = open(path+file)
        text = file.read()
        text = re.sub(r'https?:\/\/[\w\/:%#\$&\?\(\)~\.=\+\-]+',"", text) # URL삭제
        text = re.sub(r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}[+-]\d{2}\d{2}',"", text) # 시간삭제
        text = tagger.parse(text) # 띄어쓰기
        array.append(text)
        file.close()
    return tc.SFrame(data=array)

In [3]:
# IT 데이터셋 읽어오기
data = read_file('./text/it/')
data = data.rename({'X1': 'text'})
data['label'] = 0

# 스포츠 데이터셋 읽어오기
work_data = read_file('./text/sports/')
work_data = work_data.rename({'X1': 'text'})
work_data['label'] = 1

# IT와 스포츠 데이터셋 연결
data = data.append(work_data)

# 데이터셋 확인
data.explore()

# 훈련데이터 와 평가데이터 분할
train_data, test_data = data.random_split(0.8)

Materializing SFrame

In [4]:
# Bag-of-words로 변환
bow = tc.text_analytics.count_words(train_data['text'])

# Bag-of-words 확인
bow.explore()

Materializing SFrame

In [5]:
# Stop Words 읽어오기
stop_words = tc.SArray('./stop_words.txt')

# Stop Words 삭제
bow = bow.dict_trim_by_keys(stop_words, exclude=True)

# Bag-of-words 확인
bow.explore()

Finished parsing file /Users/furukawahidekazu/Documents/store/book/MLPhone/sample/ch6/TextAnalysis/stop_words.txt

Parsing completed. Parsed 104 lines in 0.034646 secs.

Materializing SFrame

In [6]:
# 모델 생성
model = tc.topic_model.create(bow)

Learning a topic model

Number of documents      1415

Vocabulary size     25215

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

| Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |

+-----------+---------------+----------------+-----------------+

| 10        | 389.602ms     | 6.96246e+06    | 0               |

+-----------+---------------+----------------+-----------------+

In [7]:
# 각 토픽별 발생횟수가 많은 단어와 점수 획득
topics = model.get_topics()

# 결과 확인
topics.explore()

Materializing SFrame

In [8]:
# 토픽별 단어리스트 표시
topics_word = model.get_topics(output_type='topic_words')

# 결과 확인
topics_word.explore()

Materializing SFrame

In [9]:
# Bag-of-words로 변환
test_bow = tc.text_analytics.count_words(test_data['text'])

# Stop Words 삭제
test_bow = test_bow.dict_trim_by_keys(stop_words, exclude=True)

# 예측
pred = model.predict(test_bow[0:1])
print(pred)

# 정답데이터
print(test_data[0:1]['text'])

[0]
["가격 대비 최고의 객실. 도로에 붙어있어서 시끄럽다는 의견이 있었으나 전혀 문제 없었음(객실 위체에 따라서 다른 듯). 리모델링 직후 호텔을 이용해서 너무 깨끗하고 안락하게 이용했음. 지하철에서도 매우 가까워서 이용하는데 큰 무리가 없으나, 주차장은 오후7~8시 이후엔 만차여서 이용하기 어려움. 대만족!!! \n"]
